In [1]:
track_progress = True
# If true, the model prints statements as it completes certain long calculations

# Specify tax reform

In [2]:
btax_dict1 = {
    2017: {
        #'tau_c': 0.357,
        #'tau_nc': 0.0,
        #'tau_amt': 0.2,
        #'pymtc_status': 0,
        #'depr_3yr_method': 'GDS',
        #'depr_3yr_bonus': 0.5,
        #'depr_5yr_method': 'GDS',
        #'depr_5yr_bonus': 0.5,
        #'depr_7yr_method': 'GDS',
        #'depr_7yr_bonus': 0.5,
        #'depr_10yr_method': 'GDS',
        #'depr_10yr_bonus': 0.5,
        #'depr_15yr_method': 'GDS',
        #'depr_15yr_bonus': 0.5,
        #'depr_20yr_method': 'GDS',
        #'depr_20yr_bonus': 0.5,
        #'depr_25yr_method': 'GDS',
        #'depr_25yr_bonus': 0.0,
        #'depr_275yr_method': 'GDS',
        #'depr_275yr_bonus': 0.0,
        #'depr_39yr_method': 'GDS',
        #'depr_39yr_bonus': 0.0,
        #'depr_land_bonus': 0.0,
        #'inventory_method': 'Mix'
    }
}
btax_dict2 = {
    #'undepBasis_corp_hc': {0: 0.0},
    #'undepBasis_noncorp_hc': {0: 0.0},
    #'ftc_hc': {9e99: 0.0},
    #'sec199_hc': {9e99: 0.0},
    #'oldIntPaid_corp_hc': {0: 0.0},
    #'newIntPaid_corp_hc': {0: 0.0},
    #'netIntPaid_corp_hc': {0: 0.0},
    #'oldIntPaid_noncorp_hc': {0: 0.0},
    #'newIntPaid_noncorp_hc': {0: 0.0},
    #'reclassify_taxdep_gdslife': {2018: {39: 25}},
    #'reclassify_taxdep_adslife': {9e99: {}}
}
iit_params_ref = {}
elast_dict = {
    #'inv_usercost_c': 0.0,
    #'inv_usercost_nc': 0.0,
    #'inv_eatr_c': 0.0,
    #'inv_eatr_nc': 0.0,
    #'mne_share_c': 0.0,
    #'mne_share_nc': 0.0,
    #'debt_taxshield_c': 0.0,
    #'debt_taxshield_nc': 0.0,
    #'legalform_ratediff': 0.0,
    #'first_year_response': 2017
}
exec(open('run_brc.py').read())

All packages successfully imported
Data imports complete
You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2013.
MTR calculations complete for 2017
MTR calculations complete for 2018
MTR calculations complete for 2019
MTR calculations complete for 2020
MTR calculations complete for 2021
MTR calculations complete for 2022
MTR calculations complete for 2023
MTR calculations complete for 2024
MTR calculations complete for 2025
MTR calculations complete for 2026
MTR calculations complete for 2027
All parameters updated
New investment matrices constructed
New corporate depreciation calculated
New noncorporate depreciation calculated
New capital paths calculated
New earnings calculated
New corporate net interest deduction calculated
New noncorporate interest deduction calculated
New taxable income calculated
Reform corporate tax revenue complete
You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded dat

# Generate the new baseline, where/when appropriate

In [4]:
exec(open('gen_baseline.py').read())

Earnings extrapolated
Investment matrices constructed
Corporate depreciation calculated
Corporate depreciation adjustment calculated
Noncorporate depreciation adjustment calculated
Noncorporate depreciation calculated
Capital paths calculated
Corporate net interest deduction calculated
Noncorporate interest deduction calculated
Taxable income calculated
You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2013.
MTR calculations complete for 2017
MTR calculations complete for 2018
MTR calculations complete for 2019
MTR calculations complete for 2020
MTR calculations complete for 2021
MTR calculations complete for 2022
MTR calculations complete for 2023
MTR calculations complete for 2024
MTR calculations complete for 2025
MTR calculations complete for 2026
MTR calculations complete for 2027
Marginal tax rates calculated
Baseline complete
BRC baseline results saved


# Check the results

### Check behavioral results

In [3]:
response_results

,Asset,deltaIc2014,deltaInc2014,MPKc2014,MPKnc2014,deltaIc2015,deltaInc2015,MPKc2015,MPKnc2015,deltaIc2016,...,MPKc2025,MPKnc2025,deltaIc2026,deltaInc2026,MPKc2026,MPKnc2026,deltaIc2027,deltaInc2027,MPKc2027,MPKnc2027
0,Aerospace products and parts manufacturing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.300471,0.317541,0.0,0.0,0.300570,0.299049,0.0,0.0,0.300570,0.298950
1,Air transportation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.138710,0.126432,0.0,0.0,0.138872,0.126778,0.0,0.0,0.138872,0.126779
2,Aircraft,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.179338,0.175009,0.0,0.0,0.179463,0.173864,0.0,0.0,0.179463,0.173858
3,"All other nonmanufacturing, n.e.c.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.240471,0.254033,0.0,0.0,0.240570,0.239030,0.0,0.0,0.240570,0.238950
4,Amusement and recreation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.121377,0.116936,0.0,0.0,0.121502,0.116410,0.0,0.0,0.121502,0.116407
5,Autos,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.447847,0.436170,0.0,0.0,0.447996,0.436376,0.0,0.0,0.447996,0.436377
6,Books,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.236590,0.223878,0.0,0.0,0.236754,0.224449,0.0,0.0,0.236754,0.224451
7,"Chemical manufacturing, ex. pharma and med",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.240471,0.254033,0.0,0.0,0.240570,0.239030,0.0,0.0,0.240570,0.238950
8,Communication,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.124331,0.116440,0.0,0.0,0.124475,0.116510,0.0,0.0,0.124475,0.116510
9,Communications,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.213685,0.206609,0.0,0.0,0.213821,0.206064,0.0,0.0,0.213821,0.206061


In [4]:
print(rescale_corp)
print(rescale_noncorp)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


### Check capital results

In [5]:
capPath_ref_corp

,year,Kstock,Investment,FixedInv,TrueDep,taxDep,FixedK
0,2014,8523.470594,1105.493182,1027.139697,477.319222,851.826463,6191.422047
1,2015,9178.808483,1092.591724,984.861573,546.203522,880.672564,6761.251890
2,2016,9841.386317,915.234108,887.220477,600.251497,852.728575,7285.929270
3,2017,10329.814397,948.935284,919.890122,636.275230,876.969610,7702.224479
4,2018,10847.898452,988.621883,958.361987,672.438252,860.876840,8139.984263
5,2019,11386.671656,1024.431225,993.075272,709.030964,852.845092,8593.903831
6,2020,11939.218032,1059.758982,1027.321711,745.425629,761.021746,9057.862509
7,2021,12503.092416,1099.676545,1066.017471,781.617843,819.283949,9529.960837
8,2022,13078.932434,1143.137205,1108.147879,818.160852,871.016417,10011.687423
9,2023,13672.081777,1188.529121,1152.150433,855.665285,920.974532,10507.780810


In [6]:
capPath_base_corp

,year,Kstock,Investment,FixedInv,TrueDep,taxDep,FixedK
0,2014,8523.470594,1105.493182,1027.139697,477.319222,851.826463,6191.422047
1,2015,9178.808483,1092.591724,984.861573,546.203522,880.672564,6761.251890
2,2016,9841.386317,915.234108,887.220477,600.251497,852.728575,7285.929270
3,2017,10329.814397,948.935284,919.890122,636.275230,876.969610,7702.224479
4,2018,10847.898452,988.621883,958.361987,672.438252,860.876840,8139.984263
5,2019,11386.671656,1024.431225,993.075272,709.030964,852.845092,8593.903831
6,2020,11939.218032,1059.758982,1027.321711,745.425629,761.021746,9057.862509
7,2021,12503.092416,1099.676545,1066.017471,781.617843,819.283949,9529.960837
8,2022,13078.932434,1143.137205,1108.147879,818.160852,871.016417,10011.687423
9,2023,13672.081777,1188.529121,1152.150433,855.665285,920.974532,10507.780810


In [7]:
capPath_ref_noncorp

,year,Kstock,Investment,FixedInv,TrueDep,taxDep,FixedK
0,2014,6682.315022,303.821534,269.090642,158.046910,254.771279,2545.791151
1,2015,6848.356794,335.238119,284.800549,168.820313,277.820019,2664.720905
2,2016,7098.582066,304.308697,290.277636,180.686200,278.668624,2813.922953
3,2017,7345.541817,315.514093,300.966374,191.476957,290.810275,2973.440729
4,2018,7613.758844,328.709600,313.553462,202.359645,292.627647,3142.437721
5,2019,7894.431160,340.615946,324.910831,213.402479,295.771263,3318.502473
6,2020,8184.205741,352.362168,336.115458,224.423093,279.024810,3499.521057
7,2021,8481.419922,365.634469,348.775800,235.417176,296.061708,3684.755035
8,2022,8784.782289,380.084822,362.559877,246.537990,311.514381,3874.478310
9,2023,9096.758788,395.177305,376.956475,257.961988,327.065676,4070.338428


In [8]:
capPath_base_noncorp

,year,Kstock,Investment,FixedInv,TrueDep,taxDep,FixedK
0,2014,6682.315022,303.821534,269.090642,158.046910,254.771279,2545.791151
1,2015,6848.356794,335.238119,284.800549,168.820313,277.820019,2664.720905
2,2016,7098.582066,304.308697,290.277636,180.686200,278.668624,2813.922953
3,2017,7345.541817,315.514093,300.966374,191.476957,290.810275,2973.440729
4,2018,7613.758844,328.709600,313.553462,202.359645,292.627647,3142.437721
5,2019,7894.431160,340.615946,324.910831,213.402479,295.771263,3318.502473
6,2020,8184.205741,352.362168,336.115458,224.423093,279.024810,3499.521057
7,2021,8481.419922,365.634469,348.775800,235.417176,296.061708,3684.755035
8,2022,8784.782289,380.084822,362.559877,246.537990,311.514381,3874.478310
9,2023,9096.758788,395.177305,376.956475,257.961988,327.065676,4070.338428


### Check combined results

In [9]:
combined_ref

,year,ebitda,taxDep,nid,sec199,taxinc,tau,taxbc,ftc,amt,pymtc,gbc,taxrev
0,2014,2397.819134,851.826463,231.441998,32.956061,1281.594612,0.347,444.713330,101.582376,4.478907,3.646970,26.950859,317.012033
1,2015,2415.401676,880.672564,243.981131,33.904093,1256.843889,0.347,436.124829,91.794692,4.392409,3.712986,27.726142,317.283418
2,2016,2412.741160,852.728575,248.001094,33.376302,1278.635190,0.347,443.686411,96.318143,4.468565,3.792439,27.294524,320.749869
3,2017,2422.110804,876.969610,268.261581,33.230911,1243.648701,0.347,431.546099,99.935946,4.346294,3.876830,27.175626,304.903992
4,2018,2426.737788,860.876840,295.578104,33.593393,1236.689452,0.347,429.131240,96.600049,4.321973,3.946732,27.472057,305.434374
5,2019,2449.641363,852.845092,328.930336,33.870235,1233.995700,0.347,428.196508,97.986091,4.312559,4.003551,27.698453,302.820972
6,2020,2484.806446,761.021746,362.442904,34.003676,1327.338120,0.347,460.586328,103.553749,4.638772,4.050054,27.807579,329.813717
7,2021,2536.281651,819.283949,393.691242,34.427901,1288.878559,0.347,447.240860,109.003947,4.504363,4.114760,28.154502,310.472014
8,2022,2605.917770,871.016417,422.511919,35.134941,1277.254493,0.347,443.207309,114.806528,4.463740,4.170710,28.732707,299.961105
9,2023,2700.770956,920.974532,448.477548,36.190523,1295.128353,0.347,449.409538,121.619274,4.526205,4.215607,29.595942,298.504920


In [10]:
combined_base

,year,ebitda,taxDep,nid,sec199,taxinc,amt,pymtc,ftc,gbc,tau,taxbc,taxrev
0,2014,2397.819134,851.826463,231.441998,32.956061,1281.594612,4.478907,3.646970,101.582376,26.950859,0.347,444.713330,317.012033
1,2015,2415.401676,880.672564,243.981131,33.904093,1256.843889,4.392409,3.712986,91.794692,27.726142,0.347,436.124829,317.283418
2,2016,2412.741160,852.728575,248.001094,33.376302,1278.635190,4.468565,3.792439,96.318143,27.294524,0.347,443.686411,320.749869
3,2017,2422.110804,876.969610,268.261581,33.230911,1243.648701,4.346294,3.876830,99.935946,27.175626,0.347,431.546099,304.903992
4,2018,2426.737788,860.876840,295.578104,33.593393,1236.689452,4.321973,3.946732,96.600049,27.472057,0.347,429.131240,305.434374
5,2019,2449.641363,852.845092,328.930336,33.870235,1233.995700,4.312559,4.003551,97.986091,27.698453,0.347,428.196508,302.820972
6,2020,2484.806446,761.021746,362.442904,34.003676,1327.338120,4.638772,4.050054,103.553749,27.807579,0.347,460.586328,329.813717
7,2021,2536.281651,819.283949,393.691242,34.427901,1288.878559,4.504363,4.114760,109.003947,28.154502,0.347,447.240860,310.472014
8,2022,2605.917770,871.016417,422.511919,35.134941,1277.254493,4.463740,4.170710,114.806528,28.732707,0.347,443.207309,299.961105
9,2023,2700.770956,920.974532,448.477548,36.190523,1295.128353,4.526205,4.215607,121.619274,29.595942,0.347,449.409538,298.504920


### Check results distributed

In [11]:
indiv_gfactors

,year,Investment,inc_aftertax_base,inc_aftertax_ref,equity,debt,SchC_pos,SchC_neg,e26270_pos,e26270_neg
0,2014,1105.493182,1372.045882,1372.045882,1.0,1.0,1.0,1.0,1.0,1.0
1,2015,1092.591724,1307.933605,1307.933605,1.0,1.0,1.0,1.0,1.0,1.0
2,2016,915.234108,1243.738700,1243.738700,1.0,1.0,1.0,1.0,1.0,1.0
3,2017,948.935284,1212.670000,1212.670000,1.0,1.0,1.0,1.0,1.0,1.0
4,2018,988.621883,1153.287059,1153.287059,1.0,1.0,1.0,1.0,1.0,1.0
5,2019,1024.431225,1108.859091,1108.859091,1.0,1.0,1.0,1.0,1.0,1.0
6,2020,1059.758982,1047.124196,1047.124196,1.0,1.0,1.0,1.0,1.0,1.0
7,2021,1099.676545,1050.500551,1050.500551,1.0,1.0,1.0,1.0,1.0,1.0
8,2022,1143.137205,1065.283894,1065.283894,1.0,1.0,1.0,1.0,1.0,1.0
9,2023,1188.529121,1098.123203,1098.123203,1.0,1.0,1.0,1.0,1.0,1.0


### Check final results

In [13]:
compare = np.asarray(combined_ref['taxrev'] - combined_base['taxrev'])

print("Total corporate tax revenue change, 2014-2027: " + str(np.round(sum(compare), 3)))
print("Total corporate tax revenue change, 2017-2026: " + str(np.round(sum(compare[3:13]), 3)))
print("Total corporate tax revenue change, 2018-2027: " + str(np.round(sum(compare[4:]), 3)))

Total corporate tax revenue change, 2014-2027: 0.0
Total corporate tax revenue change, 2017-2026: 0.0
Total corporate tax revenue change, 2018-2027: 0.0


In [14]:
ModelResults

,year,IndivTaxRev,CorpTaxRev,RevenueChange
0,2014,5.366732e-14,0.000000e+00,5.366732e-14
1,2015,8.413892e-14,0.000000e+00,8.413892e-14
2,2016,3.461534e-14,0.000000e+00,3.461534e-14
3,2017,2.049967e-13,0.000000e+00,2.049967e-13
4,2018,5.776415e-14,5.684342e-14,1.146076e-13
5,2019,-4.584188e-14,-5.684342e-14,-1.026853e-13
6,2020,-7.917743e-14,0.000000e+00,-7.917743e-14
7,2021,6.947060e-14,5.684342e-14,1.263140e-13
8,2022,0.000000e+00,5.684342e-14,5.684342e-14
9,2023,5.185575e-14,-5.684342e-14,-4.987667e-15


In [15]:
print("Total tax revenue change, 2014-2027: " + str(np.round(sum(ModelResults['RevenueChange']), 3)))
print("Total tax revenue change, 2017-2026: " + str(np.round(sum(ModelResults['RevenueChange'][3:13]), 3)))
print("Total tax revenue change, 2018-2027: " + str(np.round(sum(ModelResults['RevenueChange'][4:]), 3)))

Total tax revenue change, 2014-2027: 0.0
Total tax revenue change, 2017-2026: 0.0
Total tax revenue change, 2018-2027: 0.0
